In [1]:
# Import the functions from api_request_wrapper.py
from api_request import request_gemini as request_llm

# Define the question you want to ask
messages = [
        {"role": "system", "content": "You answer questions about web services."},
        {"role": "user", "content": "What are the main differences between classical and quantum physics1"}]
    
llm_response = request_llm(messages)
print("LLM Response:")
print(llm_response)


request gemini-1.5-flash
Received response from Gemini.
LLM Response:
I am gzm. 

